FB3_DEBERTA_FAMILY

# Directory settings

In [1]:
!pip install opendatasets
import opendatasets as od
# od.download("https://www.kaggle.com/datasets/conjuring92/fpe-tapt-delv3-span-mlm")


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install inflect
import inflect
p = inflect.engine()


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [4]:
!pip install -q bitsandbytes-cuda110
!pip install --upgrade wandb
!pip download transformers[sentencepiece]==4.18.0
!pip download tokenizers==0.12.1
!pip3 install -U scikit-learn
# !pip install --upgrade torch==1.9.0
!pip3 install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
  File was already downloaded /home/transformers-4.18.0-py3-none-any.whl
  File was already downloaded /home/tokenizers-0.12.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
  File was already downloaded /home/tqdm-4.64.1-py2.py3-none-any.whl
  File was already downloaded /home/PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl
  File was already downloaded /home/huggingface_hub-0.9.1-py3-none-any.whl
  File was already downloaded /home/packaging-21.3-py3-none-any.whl
  File was already downloaded /home/requests-2.28.1-py3-none-any.whl
  File was already downloaded /home/numpy-1.23.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  File was already downloaded /home/sacremoses-0.0.53.tar.gz
  Preparing metadata (setup.py) ... done
  File was already downloaded /home/regex-2022.9.13-cp38-cp38-manylinux_2_

# CFG

In [5]:
class CFG:
    wandb=True
    competition='FB3'
    _wandb_kernel='harshit-FB3'
    debug=False
    apex=True
    print_freq=20
    num_workers=4
    model="microsoft/deberta-v3-large"
    gradient_checkpointing=True
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=4
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=8
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_size = 6
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    train=True
    # fc_dropout = 0.0
    # fp16 = True
    freezing = False
    optim8bit = False
    gradient_checkpoint = True
    awp = True
    adv_lr = 5e-5
    adv_eps = 0.001
    awp_trigger = 0.47
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [6]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    
    import wandb

def class2dict(f):
    return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

run = wandb.init(project='feedback-public', 
                 name=CFG.model,
                 config=class2dict(CFG),
                 group=CFG.model,
                 job_type="train",
                 anonymous=None)

wandb: Currently logged in as: hmehta1992 (kaggle-clrp). Use `wandb login --relogin` to force relogin


# Library

In [7]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score, log_loss, mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from dataclasses import dataclass
import torch

print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from torch.utils.checkpoint import checkpoint
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=./ transformers')
os.system('python -m pip install --no-index --find-links=./ sentencepiece')
os.system('python -m pip install --no-index --find-links=./ tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")

from transformers import DataCollatorWithPadding
from transformers.models.deberta_v2.modeling_deberta_v2 import (ContextPooler,
                                                                StableDropout)

from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import bitsandbytes as bnb

torch.__version__: 1.8.2+cu111
Found existing installation: transformers 4.19.2
Uninstalling transformers-4.19.2:
  Successfully uninstalled transformers-4.19.2


Found existing installation: tokenizers 0.12.1
Uninstalling tokenizers-0.12.1:
  Successfully uninstalled tokenizers-0.12.1


Looking in links: ./
Processing ./transformers-4.19.2-py3-none-any.whl
Processing ./tokenizers-0.12.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


Looking in links: ./


Looking in links: ./


tokenizers.__version__: 0.12.1
transformers.__version__: 4.19.2
env: TOKENIZERS_PARALLELISM=true
==============================WARNING: DEPRECATED!==============================
WARNING! This version of bitsandbytes is deprecated. Please switch to `pip install bitsandbytes` and the new repo: https://github.com/TimDettmers/bitsandbytes
==============================WARNING: DEPRECATED!==============================


In [8]:
def freeze(module):
    """
    Freezes module's parameters.
    """
    
    for parameter in module.parameters():
        parameter.requires_grad = False
    
def set_embedding_parameters_bits(embeddings_path, optim_bits=32):
    """
    https://github.com/huggingface/transformers/issues/14819#issuecomment-1003427930
    """
    
    embedding_types = ("word", "position", "token_type")
    for embedding_type in embedding_types:
        attr_name = f"{embedding_type}_embeddings"
        
        if hasattr(embeddings_path, attr_name): 
            bnb.optim.GlobalOptimManager.get_instance().register_module_override(
                getattr(embeddings_path, attr_name), 'weight', {'optim_bits': optim_bits}
            )
            
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

# Utils

In [9]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# Data Loading

In [10]:
# ====================================================
# Data Loading
# ====================================================
INPUT_DIR = ""
# train = pd.read_csv(INPUT_DIR+'train.csv')
train = pd.read_csv(INPUT_DIR+'train.csv')
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
print(f"train.shape: {train.shape}")
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(train.head())
display(test.head())
display(submission.head())

train.shape: (3911, 8)
test.shape: (3, 2)
submission.shape: (3, 7)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [11]:
def insert_tokens(x):
    tok_map = {
        k: ["[{}_start]".format(p.number_to_words(k)), "[{}_end]".format(p.number_to_words(k))]
        for k in range(100)
    }
    appended_text = []
    for enumerate_index, row in enumerate(x):
        modified = " ".join(["[{}_start]".format(p.number_to_words(enumerate_index)),\
                              row, "[{}_end]".format(p.number_to_words(enumerate_index))])
        appended_text.append(modified)
    essay_text = "\n\n".join(appended_text)
    essay_text = "[SOE]" + essay_text + "[EOE]"
    return essay_text

# train['full_text'] = train['full_text'].apply(lambda x: x.split("\n\n"))
# train['full_text'] = train['full_text'].apply(lambda x: insert_tokens(x))

In [12]:
# ====================================================
# tokenizer
# ====================================================

tokenizer = AutoTokenizer.from_pretrained(CFG.model)
# tokenizer.add_tokens('\n')
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
# train = train.groupby('essay_id').filter(lambda x: len(x) <=13)
# train.reset_index(drop=True, inplace = True)
# train.shape

# CV split

In [14]:
os.system('pip install iterative-stratification==0.1.7')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [15]:
Fold = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train[CFG.target_cols])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    978
1    977
2    978
3    978
dtype: int64

In [16]:
train['text_length'] = train.apply(lambda x: len(x['full_text']), axis = 1)
train.sort_values(by=['text_length'], ascending=False, inplace=True)
train.reset_index(drop = True, inplace = True)

In [17]:
train.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,fold,text_length
0,7373B4F44528,Have you ever accomplished something in your l...,4.0,4.0,4.0,4.0,3.5,3.5,3,6044
1,3DCF3E6F4A13,Well most students enjoy summer vacation becau...,2.0,2.5,3.5,3.0,2.5,3.0,0,6041
2,04ACA7EB3994,"No,I dont think students should attend online ...",3.5,3.5,4.0,4.0,3.5,3.5,2,6040
3,9F68BAB4FB63,Students graduated in three year of the High S...,3.0,3.5,3.0,3.0,3.0,3.5,0,6031
4,B7FABBC13A0F,"""Character is something what you yourself choo...",3.5,3.0,3.5,3.0,3.5,3.0,0,6031


In [18]:
if CFG.debug:
    display(train.groupby('fold').size())
    train = train.sample(n=500, random_state=0).reset_index(drop=True)
    display(train.groupby('fold').size())

# Dataset

In [19]:
# ====================================================
# Define max_len
# ====================================================
lengths = []
tk0 = tqdm(train['full_text'].fillna("").values, total=len(train))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
CFG.max_len = max(lengths) + 3 # cls & sep
LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/3911 [00:00<?, ?it/s]

max_len: 1429


In [20]:
labels = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

In [21]:
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [22]:
class AWP:
    """Implements weighted adverserial perturbation
    adapted from: https://www.kaggle.com/code/wht1996/feedback-nn-train/notebook
    """

    def __init__(self, model, optimizer, adv_param="weight", adv_lr=1, adv_eps=0.001):
        self.model = model
        self.optimizer = optimizer
        self.adv_param = adv_param
        self.adv_lr = adv_lr
        self.adv_eps = adv_eps
        self.backup = {}
        self.backup_eps = {}

    def attack_backward(self, batch, labels):
        if self.adv_lr == 0:
            return
        self._save()
        self._attack_step()

        y_preds, metric, loss = self.model(batch, targets = labels)
        loss = loss.mean()
        self.optimizer.zero_grad()
        loss.backward()
        self._restore()

    def _attack_step(self):
        e = 1e-6
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                norm1 = torch.norm(param.grad)
                norm2 = torch.norm(param.data.detach())
                if norm1 != 0 and not torch.isnan(norm1):
                    r_at = self.adv_lr * param.grad / (norm1 + e) * (norm2 + e)
                    param.data.add_(r_at)
                    param.data = torch.min(
                        torch.max(param.data, self.backup_eps[name][0]), self.backup_eps[name][1]
                    )

    def _save(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                if name not in self.backup:
                    self.backup[name] = param.data.clone()
                    grad_eps = self.adv_eps * param.abs().detach()
                    self.backup_eps[name] = (
                        self.backup[name] - grad_eps,
                        self.backup[name] + grad_eps,
                    )

    def _restore(self,):
        for name, param in self.model.named_parameters():
            if name in self.backup:
                param.data = self.backup[name]
        self.backup = {}
        self.backup_eps = {}

In [23]:
# -------- Re-initialization ------------------------------------------------------#
def reinit_deberta(base_model, num_reinit_layers):
    config = base_model.config

    for layer in base_model.encoder.layer[-num_reinit_layers:]:
        for module in layer.modules():
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=config.initializer_range)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(mean=0.0, std=config.initializer_range)
                if module.padding_idx is not None:
                    module.weight.data[module.padding_idx].zero_()

In [24]:
class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.text = df['full_text'].values
        self.labels = df[CFG.target_cols].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        inputs = self.cfg.tokenizer.encode_plus(
                        self.text[item],
                        truncation=True,
                        return_token_type_ids=True,
                        add_special_tokens=True,
                        max_length=self.cfg.max_len
                    )
        samples = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
        }

        if 'token_type_ids' in inputs:
            samples['token_type_ids'] = inputs['token_type_ids']
            
        samples['target'] = self.labels[item]
            
        return samples

In [25]:
class CustomDataCollatorWithPadding(DataCollatorWithPadding):
    """
    data collector for seq classification
    """

    tokenizer = None
    padding = True
    max_length = None
    pad_to_multiple_of = None
    return_tensors = "pt"

    def __call__(self, features):
        labels = None
        if "target" in features[0].keys():
            labels = [feature["target"] for feature in features]

        batch = self.tokenizer.pad(
            features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=None,
        )

        if labels is not None:
            batch["target"] = labels

        batch = {k: (torch.tensor(v, dtype=torch.float32) if k in ["target", "aux_labels"] else torch.tensor(
            v, dtype=torch.int64)) for k, v in batch.items()}
        return batch

collate_fn = CustomDataCollatorWithPadding(tokenizer=CFG.tokenizer)

In [26]:
class FPEPooler(nn.Module):
    """Parameter free pooler to get sentence embeddings
    """

    def __init__(self, pooler_type):
        super().__init__()
        self.pooler_type = pooler_type
        assert pooler_type in ["cls", "mean", "mean_top2", "mean_first_last"]

    def forward(self, outputs, mask):
        last_hidden = outputs["last_hidden_state"]
        hidden_states = outputs["hidden_states"]

        if self.pooler_type == "cls":
            return last_hidden[:, 0]  # [CLS] output

        elif self.pooler_type == "mean":
            input_mask_expanded = mask.unsqueeze(-1).expand(last_hidden.size()).float()
            sum_embeddings = torch.sum(last_hidden * input_mask_expanded, 1)
            sum_mask = input_mask_expanded.sum(1)
            sum_mask = torch.clamp(sum_mask, min=1e-9)
            mean_embeddings = sum_embeddings / sum_mask
            return mean_embeddings

        elif self.pooler_type == "mean_first_last":
            first_hidden = hidden_states[0]
            last_hidden = hidden_states[-1]
            pooled_result = ((first_hidden + last_hidden) / 2.0 * mask.unsqueeze(-1)
                             ).sum(1) / mask.sum(-1).unsqueeze(-1)
            return pooled_result

        elif self.pooler_type == "mean_top2":
            second_last_hidden = hidden_states[-2]
            last_hidden = hidden_states[-1]
            pooled_result = ((last_hidden + second_last_hidden) / 2.0 * mask.unsqueeze(-1)
                             ).sum(1) / mask.sum(-1).unsqueeze(-1)
            return pooled_result
        else:
            raise NotImplementedError

# Model

In [27]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        self.config.hidden_dropout = 0.
        self.config.hidden_dropout_prob = 0.
        self.config.attention_dropout = 0.
        self.config.attention_probs_dropout_prob = 0.
        
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel(config=self.config)
            
        ########################### resize ############################
        self.model.resize_token_embeddings(len(cfg.tokenizer))
        # self.mean_pooler = FPEPooler(pooler_type="mean")
        # self.cls_pooler = ContextPooler(self.model.config)
        
        self.pooler = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        
#         self.normal1 = nn.LayerNorm(self.config.hidden_size*2)
#         LSTM_SIZE = self.config.hidden_size
#         self.Lstm = nn.LSTM(self.config.hidden_size, LSTM_SIZE // 2,
#                             bidirectional=True, batch_first=True)  # , num_layers=2
        
        self._init_weights(self.fc)
        
        if cfg.freezing:
                freeze(self.model.embeddings)
                freeze(self.model.encoder.layer[:2])

        # Gradient Checkpointing
        if cfg.gradient_checkpoint:
            self.model.gradient_checkpointing_enable()  
        
    def loss(self, outputs, targets):
        loss_fct = nn.MSELoss(reduction='mean')
        loss = loss_fct(outputs, targets)
        return loss
    
    def monitor_metrics(self, outputs, targets):
        device = targets.get_device()

        mll = MCRMSE(
            targets.cpu().detach().numpy(),
            outputs.cpu().detach().numpy())
        return mll
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(self, batch, targets=None):
        
        """forward pass through the model
        """
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        
        transformer_out = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True
        )
        
        last_hidden_state = transformer_out['last_hidden_state']
        
        mean_pooled_output = self.pooler(last_hidden_state, attention_mask)
        # cls_pooled_output = self.cls_pooler(transformer_out["last_hidden_state"])
        # cls_pooled_output = self.fc_dropout(cls_pooled_output)
        # sequence_output = torch.cat([mean_pooled_output, cls_pooled_output], dim=1)
        
        ######################## hidden state ###################
#         sequence_output = transformer_out[0]
        ############################ mean of all tokens ####################
        
        # sequence_output_mean = torch.mean(transformer_out[0], dim=1)
        # sequence_output_max = torch.max(transformer_out[0], dim=1)
        
#         self.Lstm.flatten_parameters()
#         lstm_output, _ = self.Lstm(sequence_output)
#         sequence_output = torch.cat((sequence_output, lstm_output), 2)
#         sequence_output = self.normal1(sequence_output)
#         sequence_output = sequence_output[:,0,:]
        
        ########################## concat ############################
        # sequence_output = torch.cat((sequence_output_mean, sequence_output_max, sequence_output), dim=1)
        
        # sequence_output = transformer_out.last_hidden_state
        # sequence_output = self.pooler(sequence_output, mask)
        # sequence_output = self.fc_dropout(sequence_output)

        # Main task
        logits = self.fc(mean_pooled_output)

        if targets is not None:
            metric = self.monitor_metrics(logits, targets)
            
            loss = self.loss(logits, targets)
            
            return logits, metric, loss
        
        return logits, 0., 0

# Helpler functions

In [28]:
AWP_FLAG = False

In [29]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    
    if CFG.awp:    
        print('Enable AWP')
        awp = AWP(model, optimizer, adv_lr = CFG.adv_lr, adv_eps = CFG.adv_eps)
        assert CFG.gradient_accumulation_steps == 1, "Grad accumulation not supported with AWP"
    
    for step, samples in enumerate(train_loader):
        labels = samples['target'].to(device, dtype = torch.float32)
        batch_size = labels.size(0)
        
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds, metric, loss = model(samples, targets = labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        if AWP_FLAG:
            print('attacking')
            awp.attack_backward(samples, labels)
        
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg

def valid_fn(valid_loader, model, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, samples in enumerate(valid_loader):
        labels = samples['target'].to(device, dtype = torch.float)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds, metric, loss = model(samples, targets = labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [30]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,collate_fn = collate_fn,
                              shuffle=True,num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,collate_fn = collate_fn,
                              shuffle=False,num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path = None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay) and p.requires_grad],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay) and p.requires_grad],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n and p.requires_grad],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    if CFG.gradient_checkpoint:
        optimizer = bnb.optim.AdamW(optimizer_parameters, lr=CFG.encoder_lr, optim_bits=8)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    best_score = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, device)
        
        # scoring
        score = get_score(valid_labels, predictions)[0]

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": avg_loss, 
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})
        
        if best_score > score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
            
        if (CFG.awp) & ( best_score <= CFG.awp_trigger):
            print("AWP is triggered...")
            AWP_FLAG = True

    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[["pred_" + c for c in labels]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [31]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        true = oof_df[CFG.target_cols].values
        preds = oof_df[["pred_" + c for c in labels]].values
        score = get_score(true, preds)[0]
        LOGGER.info(f'Score: {score:<.4f}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        
    if CFG.wandb:
        wandb.finish()

========== fold: 0 training ==========
Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

Enable AWP
Epoch: [1][0/366] Elapsed 0m 3s (remain 19m 14s) Loss: 10.1917(10.1917) Grad: inf  LR: 0.00002000  
Epoch: [1][20/366] Elapsed 0m 34s (remain 9m 23s) Loss: 0.4879(4.2712) Grad: 33850.6875  LR: 0.00001999  
Epoch: [1][40/366] Elapsed 1m 8s (remain 9m 1s) Loss: 0.3361(2.3648) Grad: 12560.1680  LR: 0.00001996  
Epoch: [1][60/366] Elapsed 1m 33s (remain 7m 46s) Loss: 0.2055(1.6835) Grad: 9609.3779  LR: 0.00001991  
Epoch: [1][80/366] Elapsed 2m 3s (remain 7m 15s) Loss: 0.2418(1.3451) Grad: 15006.6885  LR: 0.00001985  
Epoch: [1][100/366] Elapsed 2m 36s (remain 6m 49s) Loss: 0.1663(1.1336) Grad: 8219.6982  LR: 0.00001977  
Epoch: [1][120/366] Elapsed 3m 8s (remain 6m 22s) Loss: 0.3977(1.0006) Grad: 27848.4570  LR: 0.00001967  
Epoch: [1][140/366] Elapsed 3m 43s (remain 5m 56s) Loss: 0.3617(0.8999) Grad: 29621.0078  LR: 0.00001955  
Epoch: [1][160/366] Elapsed 4m 12s (remain 5m 20s) Loss: 0.3841(0.8238) Grad: 32502.8945  LR: 0.00001941  
Epoch: [1][180/366] Elapsed 4m 40s (remain 

Epoch 1 - avg_train_loss: 0.5015  avg_val_loss: 0.2170  time: 604s
Epoch 1 - Score: 0.4647
Epoch 1 - Save Best Score: 0.4647 Model


EVAL: [120/123] Elapsed 0m 37s (remain 0m 0s) Loss: 0.1363(0.2175) 
EVAL: [122/123] Elapsed 0m 38s (remain 0m 0s) Loss: 0.1614(0.2170) 
AWP is triggered...
Enable AWP
Epoch: [2][0/366] Elapsed 0m 0s (remain 5m 52s) Loss: 0.1642(0.1642) Grad: 240960.8906  LR: 0.00001706  
Epoch: [2][20/366] Elapsed 0m 32s (remain 8m 49s) Loss: 0.2808(0.2275) Grad: 520041.6250  LR: 0.00001675  
Epoch: [2][40/366] Elapsed 1m 3s (remain 8m 24s) Loss: 0.2500(0.2372) Grad: 507427.9062  LR: 0.00001643  
Epoch: [2][60/366] Elapsed 1m 33s (remain 7m 48s) Loss: 0.1985(0.2202) Grad: 453241.8438  LR: 0.00001610  
Epoch: [2][80/366] Elapsed 2m 6s (remain 7m 25s) Loss: 0.1342(0.2075) Grad: 235912.0625  LR: 0.00001575  
Epoch: [2][100/366] Elapsed 2m 42s (remain 7m 6s) Loss: 0.2679(0.2066) Grad: 384661.1562  LR: 0.00001540  
Epoch: [2][120/366] Elapsed 3m 12s (remain 6m 29s) Loss: 0.1589(0.2025) Grad: 255898.6406  LR: 0.00001503  
Epoch: [2][140/366] Elapsed 3m 51s (remain 6m 9s) Loss: 0.2105(0.1965) Grad: 586213.812

Epoch 2 - avg_train_loss: 0.1846  avg_val_loss: 0.2049  time: 624s
Epoch 2 - Score: 0.4519
Epoch 2 - Save Best Score: 0.4519 Model


...

AWP is triggered...
Enable AWP
Epoch: [4][0/366] Elapsed 0m 1s (remain 8m 56s) Loss: 0.1353(0.1353) Grad: 206138.5625  LR: 0.00000294  
Epoch: [4][20/366] Elapsed 0m 35s (remain 9m 48s) Loss: 0.1631(0.1597) Grad: 129416.5781  LR: 0.00000264  
Epoch: [4][40/366] Elapsed 1m 0s (remain 7m 58s) Loss: 0.1335(0.1578) Grad: 132715.2500  LR: 0.00000236  
Epoch: [4][60/366] Elapsed 1m 37s (remain 8m 7s) Loss: 0.1828(0.1567) Grad: 98651.5547  LR: 0.00000209  
Epoch: [4][80/366] Elapsed 2m 12s (remain 7m 45s) Loss: 0.1546(0.1544) Grad: 91023.1953  LR: 0.00000183  
Epoch: [4][100/366] Elapsed 2m 47s (remain 7m 19s) Loss: 0.1715(0.1550) Grad: 143342.1719  LR: 0.00000159  
Epoch: [4][120/366] Elapsed 3m 20s (remain 6m 46s) Loss: 0.0999(0.1549) Grad: 103660.1953  LR: 0.00000137  
Epoch: [4][140/366] Elapsed 3m 54s (remain 6m 13s) Loss: 0.1915(0.1535) Grad: 146539.9688  LR: 0.00000116  
Epoch: [4][160/366] Elapsed 4m 25s (remain 5m 38s) Loss: 0.2383(0.1537) Grad: 127553.3438  LR: 0.00000097  
Epoch: [

Epoch 4 - avg_train_loss: 0.1539  avg_val_loss: 0.1984  time: 645s
Epoch 4 - Score: 0.4448
Epoch 4 - Save Best Score: 0.4448 Model


AWP is triggered...


========== fold: 3 result ==========
Score: 0.4448
========== CV ==========
Score: 0.4526


[fold0] avg_train_loss,█▁▁▁
[fold0] avg_val_loss,█▂▁▁
[fold0] epoch,▁▃▆█
[fold0] loss,█▃▆▅█▄▇▅▄▂▂▃▂▂▃▂▃▄▃▇▃▂▃▄▂▂▃▄▂▄▄▃▅▄▁▃▄▂▃▂
[fold0] lr,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
[fold0] score,█▂▁▁
[fold1] avg_train_loss,█▂▁▁
[fold1] avg_val_loss,█▃▁▁
[fold1] epoch,▁▃▆█
[fold1] loss,▆▅▄▃▄▃▃▃▄▃█▂▂▃▂▃▂▂▂▂▁▂▃▄▂▂▂▂▂▂▃▂▁▂▂▂▃▃▂▄
[fold1] lr,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁


In [32]:
import pandas as pd
pd.read_pickle('oof_df.pkl').to_csv('oof_df.csv', index = False)